In [3]:
import ccxt
import pandas as pd

In [ ]:
api_keys = { "phemex": 
    { "apiKey": "#",
      "secret": "#",
        "enableRateLimit": True,
          "options": {"defaultType": "swap"}
}}
phemex = ccxt.phemex(api_keys['phemex'])
markets = phemex.load_markets()

In [5]:
funding = []

for symbol, market in phemex.markets.items():

    if market.get('swap') and market.get('contract'):

        market_id = market['id']
        try:
            rate = phemex.fetch_funding_rate(symbol)
            funding.append(rate)
        except Exception as e:
            print("Skipping", symbol, "| ID:", market_id, "| Error:", e)

df_funndig = pd.DataFrame(funding)

Skipping ETH/USD:USD | ID: ETHUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping XRP/USD:USD | ID: XRPUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping LINK/USD:USD | ID: LINKUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping XTZ/USD:USD | ID: XTZUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping LTC/USD:USD | ID: LTCUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping GOLD/USD:USD | ID: GOLDUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping ADA/USD:USD | ID: ADAUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":null,"result":null}
Skipping BCH/USD:USD | ID: BCHUSD | Error: phemex {"error":{"code":6001,"message":"invalid argument"},"id":

In [88]:
df_funndig['fundingRate'] = pd.to_numeric(df_funndig['fundingRate'], errors='coerce')
df_funndig = df_funndig.sort_values(by='fundingRate')
tikcer_names = pd.concat([df_funndig['symbol'].head(10),df_funndig['symbol'].tail(10)])

In [89]:
long_names = df_funndig[['symbol']].head(10)
short_names= df_funndig[['symbol']].tail(10)

In [ ]:
funding_list = []

for ticker in tikcer_names:
    try:
        funding_data = phemex.fetch_funding_rate_history(ticker)
        df_f = pd.DataFrame(funding_data)
        if 'timestamp' in df_f.columns:
            df_f['timestamp'] = pd.to_datetime(df_f['timestamp'], unit='ms')
        df_f['symbol'] = ticker
        funding_list.append(df_f)
    except Exception as e:
        print(f"Skipping {ticker} funding history | Error: {e}")

df_funding = pd.concat(funding_list, ignore_index=True)

numeric_cols = ['fundingRate', 'funding', 'rate']
for col in numeric_cols:
    if col in df_funding.columns:
        df_funding[col] = pd.to_numeric(df_funding[col], errors='coerce')

In [32]:
df_2 = df_funding.copy()
interval_map = {}

for symbol, group in df_2.groupby('symbol'):
    td = group['timestamp'].iloc[1] - group['timestamp'].iloc[0]
    hours = td.total_seconds() / 3600
    interval_map[symbol] = hours

df_2['interval'] = df_2['symbol'].map(interval_map)

df_2['2_days_moving_avr'] = (
    df_2.groupby('symbol')
        .apply(lambda g: g['fundingRate'].ewm(span=48 / g['interval'].iloc[0]).mean())
        .reset_index(level=0, drop=True)
)


C:\Users\wkaco\AppData\Local\Temp\ipykernel_11652\4226331411.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g['fundingRate'].ewm(span=48 / g['interval'].iloc[0]).mean())


In [105]:
# Create empty results DataFrame
residuals = pd.DataFrame(index=["r2", "mse", "accuracy"], columns=tikcer_names)

for symbol in tikcer_names:

    # Filter for this symbol
    df_sym = df_2[df_2["symbol"] == symbol]

    # Last 10 rows
    df_last = df_sym
        # --- R2 ---
    r2 = (
        ((df_last['2_days_moving_avr'].shift(1).dropna() - df_last['fundingRate'].mean()) ** 2).sum()
        /
        ((df_last['fundingRate'] - df_last['fundingRate'].mean()) ** 2).sum()
    )

    # --- MSE ---
    mse = ((df_last['2_days_moving_avr'].shift(1) - df_last['fundingRate']) ** 2).mean()

    # --- Accuracy of sign ---
    sign_ma = df_last['2_days_moving_avr'].shift(1).apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
    sign_funding = df_last['fundingRate'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)

    matches = (sign_ma == sign_funding).sum()
    accuracy = matches / len(df_last)

    # Save results
    residuals.loc["r2", symbol] = r2
    residuals.loc["mse", symbol] = mse
    residuals.loc["accuracy", symbol] = accuracy

# Final table
residuals = residuals.loc[:, residuals.loc["accuracy"] >= 0.8]

In [161]:
balance = phemex.fetch_balance()
balance_for_shorts = balance['total']["USDT"] / 2
balance_for_longs = balance['total']["USDT"] / 2

In [169]:
# 1. Extract the BUY/SELL row
signals = residuals.iloc[3]

# 2. Boolean mask for BUY signals
is_buy = signals == "BUY"

# 3. Count BUY and SELL
count_buy = is_buy.sum()
count_sell = (~is_buy).sum()

# 5. Assign weights based on BUY/SELL
residuals.loc["weight", is_buy] = 1 / count_buy * balance_for_longs - 1
residuals.loc["weight", ~is_buy] = 1 / count_sell * balance_for_shorts - 1


In [261]:
def place_residual_orders():
    """Place limit orders based on residuals DataFrame"""
    
    for ticker in residuals.columns:
        try:
            # Extract order details
            side = residuals[ticker][3]  # "BUY" or "SELL"
            amount = residuals[ticker][4]
            
            # Skip if no amount to trade
            if amount == 0:
                print(f"Skipping {ticker} - zero amount")
                continue
            
            # Set leverage
            phemex.set_leverage(symbol=ticker, leverage=5)
            
            # Fetch order book and determine price
            order_book = phemex.fetch_order_book(ticker)
            price = order_book["bids"][0][0] if side == "BUY" else order_book["asks"][0][0]
            
            # Place order using unified method
            order = phemex.create_order(
                symbol=ticker,
                type="limit",
                side=side.lower(),  # "buy" or "sell"
                amount=amount * 5 / price,  # Use actual amount instead of hardcoded 1
                price=price
            )
            
            print(f"Order placed → {ticker} | {side} | amount: {amount} | price: {price}")
            
        except Exception as e:
            print(f"Error placing order for {ticker}: {e}")
            continue

# Call the function
place_residual_orders()

C:\Users\wkaco\AppData\Local\Temp\ipykernel_11652\3276846832.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  side = residuals[ticker][3]  # "BUY" or "SELL"
C:\Users\wkaco\AppData\Local\Temp\ipykernel_11652\3276846832.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  amount = residuals[ticker][4]


Order placed → PLUME/USDT:USDT | BUY | amount: 11.237625 | price: 0.0269
Order placed → TNSR/USDT:USDT | BUY | amount: 11.237625 | price: 0.1478
Order placed → VELO/USDT:USDT | BUY | amount: 11.237625 | price: 0.005552
Order placed → TRUST/USDT:USDT | BUY | amount: 11.237625 | price: 0.1707
Order placed → XPIN/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.002285
Order placed → SAHARA/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.07991
Order placed → ALCH/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.14703
Order placed → IDOL/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.03193
Order placed → B/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.1948
Order placed → 4/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.0315
Order placed → ARC/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.02807
Order placed → XNY/USDT:USDT | SELL | amount: 4.438944444444444 | price: 0.003947
Error placing order for DOCK/USDT:USDT: phemex {"code":399

In [260]:
def cancel_all_orders():
    for ticker in residuals.columns:
        open_orders = phemex.fetch_open_orders(symbol=ticker)

        for order in open_orders:
            try:
                phemex.cancel_order(order['id'], order['symbol'])
                print(f"Cancelled → {order['symbol']} | id: {order['id']}")
            except Exception as e:
                print(f"Error cancelling {order['id']}: {e}")


def set_stop_loss_take_profit_for_positions():
    """Set stop-loss (-0.1%) and take-profit (+0.1%) orders for all open positions"""
    positions = phemex.fetch_positions()

    for pos in positions:
        contracts = float(pos['contracts'])
        side = pos['side']  # 'long' or 'short'
        symbol = pos['symbol']

        if contracts == 0:
            continue  # empty position

        try:
            # Fetch current market price
            ticker = phemex.fetch_ticker(symbol)
            current_price = ticker['last']  # Current market price
            
            # Calculate stop-loss and take-profit prices based on position type
            if side == "long":
                # For long: TP above current, SL below current
                take_profit_price = current_price * 1.001  # +0.1%
                stop_loss_price = current_price * 0.999    # -0.1%
                close_side = "sell"
            else:  # short
                # For short: TP below current, SL above current
                take_profit_price = current_price * 0.999  # -0.1%
                stop_loss_price = current_price * 1.001    # +0.1%
                close_side = "buy"
            
            # Place Take Profit order (limit order to close position at profit)
            try:
                tp_order = phemex.create_order(
                    symbol=symbol,
                    type="limit",
                    side=close_side,
                    amount=contracts,
                    price=take_profit_price,
                    params={
                        "reduceOnly": True
                    }
                )
                print(f"✓ TP set → {symbol} | {side} | TP: {take_profit_price:.6f} (+0.1%)")
            except Exception as e:
                print(f"✗ Error setting TP for {symbol}: {e}")
            
            # Place Stop Loss order (conditional market order)
            try:
                sl_order = phemex.create_order(
                    symbol=symbol,
                    type="market",
                    side=close_side,
                    amount=contracts,
                    params={
                        "trigger": stop_loss_price,  # Use trigger instead of stopLoss object
                        "reduceOnly": True
                    }
                )
                print(f"✓ SL set → {symbol} | {side} | SL: {stop_loss_price:.6f} (-0.1%)")
            except Exception as e:
                print(f"✗ Error setting SL for {symbol}: {e}")
                
        except Exception as e:
            print(f"✗ Error processing {symbol}: {e}")


def protect_all_positions():
    print("Cancelling all open orders...")
    cancel_all_orders()

    print("\nSetting stop-loss and take-profit orders...")
    set_stop_loss_take_profit_for_positions()

    print("\nAll positions protected with SL/TP orders.")

In [256]:
protect_all_positions()

Cancelling all open orders...

Setting stop-loss and take-profit orders...
✓ TP set → XPIN/USDT:USDT | short | TP: 0.002302 (+0.1%)
✗ Error setting SL for XPIN/USDT:USDT: phemex {"code":11140,"msg":"TE_POS_REDUCE_CANNOT_ATTACH_TP_SL","data":null}
✓ TP set → ALCH/USDT:USDT | short | TP: 0.146683 (+0.1%)
✗ Error setting SL for ALCH/USDT:USDT: phemex {"code":11140,"msg":"TE_POS_REDUCE_CANNOT_ATTACH_TP_SL","data":null}

All positions protected with SL/TP orders.


In [251]:
phemex.create_order(symbol="ALCH/USDT:USDT",side="BUY",type="limit", amount=1,price= 0.14677 * .99, params={"takeProfit": 0.14677 * .99})

InvalidOrder: phemex createOrder() requires a trigger price in params["takeProfit"]["triggerPrice"] for a take profit order

In [241]:
residuals

symbol,PLUME/USDT:USDT,TNSR/USDT:USDT,VELO/USDT:USDT,TRUST/USDT:USDT,XPIN/USDT:USDT,SAHARA/USDT:USDT,ALCH/USDT:USDT,IDOL/USDT:USDT,B/USDT:USDT,4/USDT:USDT,ARC/USDT:USDT,XNY/USDT:USDT,DOCK/USDT:USDT
r2,0.01,0.568268,0.516274,0.526631,0.448603,0.138092,0.557461,0.442271,0.34871,0.085807,0.249667,0.351156,0.99
mse,0.000004,0.000016,0.0,0.000016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accuracy,0.98,0.92,0.95,0.92,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99,0.99
position,BUY,BUY,BUY,BUY,SELL,SELL,SELL,SELL,SELL,SELL,SELL,SELL,SELL
weight,11.237625,11.237625,11.237625,11.237625,4.438944,4.438944,4.438944,4.438944,4.438944,4.438944,4.438944,4.438944,4.438944
